In [ ]:
import pandas as pd
import numpy as np
import random
import math
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


# =========================
#  Device & basic config
# =========================

use_cuda = True  # set True if you want GPU and have it available
device = torch.device("cuda" if torch.cuda.is_available() and use_cuda else "cpu")
print("Using device:", device)

seeds = [i for i in range(4)]
steps_list = [2, 5, 10]
symbol_list = ['F','TGT','EL','AAPL','META','MSFT','GOOG','NVDA','AMZN']
#symbol_list = ['NVDA']
lambda_gp = 0.5  # gradient penalty coefficient


# =========================
#  Utility functions
# =========================

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)


def sliding_window_torch(x, y, window):
    """
    x: numpy array (N, n_features)
    y: numpy array (N, 1)
    Returns:
      x_: (N-window, window, n_features)
      y_: (N-window, 1)
      y_gan: (N-window, window+1, 1)
    """
    x_, y_, y_gan = [], [], []
    for i in range(window, x.shape[0]):
        tmp_x = x[i - window: i, :]      # (window, n_features)
        tmp_y = y[i]                     # (1,)
        tmp_y_gan = y[i - window: i + 1] # (window+1, 1)

        x_.append(tmp_x)
        y_.append(tmp_y)
        y_gan.append(tmp_y_gan)

    x_ = torch.tensor(np.array(x_, dtype=np.float32))
    y_ = torch.tensor(np.array(y_, dtype=np.float32))
    y_gan = torch.tensor(np.array(y_gan, dtype=np.float32))
    return x_, y_, y_gan


# =========================
#  Keras-like Multi-Head Attention
#  (to emulate TF MultiHeadAttention(num_heads, key_dim=d_model))
# =========================

class MultiHeadSelfAttentionKerasLike(nn.Module):
    """
    Emulates tf.keras.layers.MultiHeadAttention(num_heads, key_dim)
    with default output_dim = input_dim (query last dim).
    """
    def __init__(self, num_heads, key_dim, input_dim, dropout_rate=0.0):
        super().__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.input_dim = input_dim
        self.inner_dim = num_heads * key_dim  # num_heads * key_dim

        # Dense layers for Q, K, V (input_dim -> inner_dim)
        self.q_proj = nn.Linear(input_dim, self.inner_dim)
        self.k_proj = nn.Linear(input_dim, self.inner_dim)
        self.v_proj = nn.Linear(input_dim, self.inner_dim)

        # Output projection back to input_dim (like Keras default output_shape)
        self.out_proj = nn.Linear(self.inner_dim, input_dim)

        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        """
        x: (batch, seq_len, input_dim)
        returns: (batch, seq_len, input_dim)
        """
        B, T, _ = x.shape

        # Project to Q, K, V: (B, T, inner_dim)
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)

        # Reshape to (B, num_heads, T, key_dim)
        q = q.view(B, T, self.num_heads, self.key_dim).transpose(1, 2)  # (B, H, T, D)
        k = k.view(B, T, self.num_heads, self.key_dim).transpose(1, 2)
        v = v.view(B, T, self.num_heads, self.key_dim).transpose(1, 2)

        # Scaled dot-product attention
        scale = math.sqrt(self.key_dim)
        scores = torch.matmul(q, k.transpose(-1, -2)) / scale  # (B, H, T, T)
        attn_weights = torch.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Weighted sum
        context = torch.matmul(attn_weights, v)  # (B, H, T, D)

        # Back to (B, T, H*D)
        context = context.transpose(1, 2).contiguous().view(B, T, self.inner_dim)

        # Final projection to input_dim
        out = self.out_proj(context)  # (B, T, input_dim)
        return out


# =========================
#  Transformer encoder2 (PyTorch version)
#  Matches your TF transformer_encoder2
# =========================

class TransformerEncoder2(nn.Module):
    """
    PyTorch equivalent of your TF transformer_encoder2:

    def transformer_encoder2(inputs, d_model, num_heads, dff, dropout_rate=0.0):
        x = LayerNormalization(epsilon=1e-6)(inputs)
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x, x)
        attn_output = Dropout(dropout_rate)(attn_output)
        x = LayerNormalization(epsilon=1e-6)(x + attn_output)

        ffn_output = Sequential([Dense(dff, relu), Dense(d_model)])(x)
        ffn_output = Dropout(dropout_rate)(ffn_output)
        x = LayerNormalization(epsilon=1e-6)(x + ffn_output)

        x = reduce_mean(x, axis=1)
        split into num_heads chunks along last dim, stack, mean => (batch, d_model/num_heads)
    """

    def __init__(self, d_model, num_heads, dff, input_dim=1, dropout_rate=0.0):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.input_dim = input_dim
        self.dff = dff
        self.dropout_rate = dropout_rate

        # First LayerNorm over inputs
        self.ln1 = nn.LayerNorm(input_dim, eps=1e-6)

        # Keras-like multi-head self-attention
        self.mha = MultiHeadSelfAttentionKerasLike(
            num_heads=num_heads,
            key_dim=d_model,
            input_dim=input_dim,
            dropout_rate=dropout_rate
        )
        self.dropout_attn = nn.Dropout(dropout_rate)
        self.ln2 = nn.LayerNorm(input_dim, eps=1e-6)

        # FFN: Dense(dff, relu) -> Dense(d_model)
        self.ffn_dense1 = nn.Linear(input_dim, dff)
        self.ffn_dense2 = nn.Linear(dff, d_model)
        self.dropout_ffn = nn.Dropout(dropout_rate)
        # After x + ffn_output, last dim becomes d_model
        self.ln3 = nn.LayerNorm(d_model, eps=1e-6)

    def forward(self, x):
        """
        x: (batch, seq_len, 1)  (Close price windows scaled)
        """
        # LayerNorm before attention (over last dim)
        x_norm = self.ln1(x)  # (B, T, 1)

        # Multi-head self-attention (returns shape (B, T, 1))
        attn_out = self.mha(x_norm)
        attn_out = self.dropout_attn(attn_out)

        # Residual 1 + LayerNorm
        x2 = self.ln2(x_norm + attn_out)  # (B, T, 1)

        # FFN: shape becomes (B, T, d_model)
        ffn_hidden = torch.relu(self.ffn_dense1(x2))       # (B, T, dff)
        ffn_out = self.ffn_dense2(ffn_hidden)              # (B, T, d_model)
        ffn_out = self.dropout_ffn(ffn_out)

        # Residual 2 with broadcasting: (B, T, 1) + (B, T, d_model)
        # -> (B, T, d_model)
        # PyTorch will broadcast last dimension
        x3 = x2 + ffn_out
        x3 = self.ln3(x3)  # (B, T, d_model)

        # Mean over time axis
        x_mean = x3.mean(dim=1)  # (B, d_model)

        # Split into num_heads chunks along feature dimension and average
        chunks = torch.chunk(x_mean, self.num_heads, dim=1)   # list of num_heads tensors, each (B, d_model/num_heads)
        stacked = torch.stack(chunks, dim=0)                  # (num_heads, B, d_model/num_heads)
        output = stacked.mean(dim=0)                          # (B, d_model/num_heads)

        return output.float()


# =========================
#  Generator (PyTorch version)
#  Mirrors your TF Generator
# =========================

class Generator(nn.Module):
    """
    GRU-based Generator (paper-faithful)

    Architecture:
    GRU(input_size → 1024)
    GRU(1024 → 512)
    GRU(512 → 256)
    FC: 256 → 128 → 64 → 1
    """

    def __init__(self, input_size, dropout=0.2):
        super().__init__()

        self.gru1 = nn.GRU(input_size, 1024, batch_first=True)
        self.gru2 = nn.GRU(1024, 512, batch_first=True)
        self.gru3 = nn.GRU(512, 256, batch_first=True)

        self.dropout = nn.Dropout(dropout)

        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

        self.relu = nn.ReLU()

    def forward(self, x):
        """
        x: (batch, steps, n_features)
        """

        out, _ = self.gru1(x)
        out = self.dropout(out)

        out, _ = self.gru2(out)
        out = self.dropout(out)

        out, _ = self.gru3(out)
        out = self.dropout(out)

        # last time step
        out = out[:, -1, :]  # (batch, 256)

        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)

        return out



# =========================
#  Discriminator (PyTorch version)
#  Mirrors your TF Discriminator
# =========================

class Discriminator(nn.Module):
    """
    TF Discriminator:

    Conv1D(32) -> LeakyReLU
    Conv1D(64) -> LeakyReLU
    Conv1D(128) -> LeakyReLU
    Flatten -> Dense(220) -> LeakyReLU -> Dense(220, relu) -> Dense(1)
    """

    def __init__(self, sequence_length):
        """
        sequence_length: time dimension T = steps + 1
        input channels = 1 (since y has shape (...,1))
        """
        super().__init__()

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32,
                               kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64,
                               kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128,
                               kernel_size=5, stride=1, padding=2)

        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(128 * sequence_length, 220)
        self.linear2 = nn.Linear(220, 220)
        self.linear3 = nn.Linear(220, 1)

        self.leaky_relu = nn.LeakyReLU(0.01)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        x: (batch, T, 1)
        Conv1d expects (batch, channels=1, T)
        """
        x = x.permute(0, 2, 1)  # (B, 1, T)
        x = self.leaky_relu(self.conv1(x))
        x = self.leaky_relu(self.conv2(x))
        x = self.leaky_relu(self.conv3(x))

        x = self.flatten(x)
        x = self.leaky_relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        out = self.linear3(x)
        return out


# =========================
#  Gradient Penalty (WGAN-GP)
# =========================

def gradient_penalty(discriminator, real_samples, fake_samples, device):
    """
    real_samples: (batch, T, C)
    fake_samples: (batch, T, C)
    """
    batch_size = real_samples.size(0)

    epsilon = torch.rand(batch_size, 1, 1, device=device)
    interpolates = epsilon * real_samples + (1.0 - epsilon) * fake_samples
    interpolates.requires_grad_(True)

    critic_interpolates = discriminator(interpolates)

    grad_outputs = torch.ones_like(critic_interpolates, device=device)

    grads = torch.autograd.grad(
        outputs=critic_interpolates,
        inputs=interpolates,
        grad_outputs=grad_outputs,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]

    grads = grads.view(batch_size, -1)
    grad_norm = grads.norm(2, dim=1)
    gp = ((grad_norm - 1.0) ** 2).mean()
    return gp


# =========================
#  Main training loops (symbols, steps, seeds)
# =========================



for symbol in symbol_list:
    for steps in steps_list:
        # For each (symbol, steps), we store per-seed results to a CSV
        re_train_error = []
        re_train_acc = []
        re_testing_error = []
        re_testing_acc = []

        for seed in seeds:
            print(f"\n=== Symbol: {symbol}, steps: {steps}, seed: {seed} ===")
            set_seed(seed)

            # -------------------------
            # Data preparation
            # -------------------------
            data = pd.read_csv(f'../../Datasets/data{symbol}.csv')

            # set index
            data.index = data['Date']
            data.drop(columns=['Date'], inplace=True)

            # remove columns with infinity
            columns_infinity = data.columns.to_series()[np.isinf(data).any()]
            data.drop(columns=columns_infinity, inplace=True)

            # select features (exactly as your TF code)
            data = data[['Close', 'pct_change',
                         'log_change', '7sma', '14sma', '21sma', '7ema', '14ema', '21ema',
                         '7macd', '14macd', '7rsi', '14rsi', '21rsi', '7atr', '14atr', '21atr',
                         '7upper', '7lower', '14upper', '14lower', '21upper', '21lower', '7rsv',
                         '14rsv', '21rsv', '3compo_FT', '6compo_FT', '9compo_FT', '27compo_FT',
                         '81compo_FT', '100compo_FT']]

            # prepare for data enhancement
            data_transform = data['Close']
            data_transform = np.array(data_transform).reshape(-1, 1)
            data['y'] = data['Close']
            data = data.drop(columns=['Close'])
            x_later = data.iloc[:, :-1].values  # features except y
            y_later = data.iloc[:, -1].values   # y

            # scale Close for transformer feature extraction
            #scaler_close = MinMaxScaler(feature_range=(0, 1))
            #data_scaled = scaler_close.fit_transform(np.array(data_transform).reshape(-1, 1))

            # look back data for feature extraction
            """
            time_step = 3
            #X, y_seq = create_dataset(data_scaled, time_step)
            X, y_seq = create_dataset(data_transform, time_step)
            X = X.reshape(X.shape[0], X.shape[1], 1)  # (N, time_step, 1)

            # attention-based feature extraction
            sequence_length = time_step
            features = 1

            d_model = 256
            heads = 4
            dff = 6
            dropout_rate = 0.0

            transformer_model = TransformerEncoder2(
                d_model=d_model,
                num_heads=heads,
                dff=dff,
                input_dim=features,
                dropout_rate=dropout_rate
            ).to(device)

            transformer_model.eval()
            with torch.no_grad():
                X_tensor = torch.tensor(X, dtype=torch.float32, device=device)
                extracted_features_torch = transformer_model(X_tensor)  # (N, d_model/num_heads=64)
                extracted_features = extracted_features_torch.cpu().numpy()

            # combine extracted feature with original
            m = x_later.shape[0]
            n = extracted_features.shape[0]
            x_later = x_later[m - n:]  # align lengths
            x_later = np.hstack((x_later.astype(np.float32),
                                 extracted_features.astype(np.float32)))
            y_later = y_later[m - n:]
            y_later = y_later.astype(np.float32)
            
            """
            # split dataset & scale
            split = int(x_later.shape[0] * 0.8)
            train_x, test_x = x_later[:split, :], x_later[split - 3:, :]
            train_y, test_y = y_later[:split], y_later[split - 3:]

            x_scaler = MinMaxScaler(feature_range=(0, 1))
            y_scaler = MinMaxScaler(feature_range=(0, 1))
            train_x = x_scaler.fit_transform(train_x)
            test_x = x_scaler.transform(test_x)
            train_y_scaled = y_scaler.fit_transform(train_y.reshape(-1, 1))
            test_y_scaled = y_scaler.transform(test_y.reshape(-1, 1))

            # sliding window for WGAN-GP
            train_x_slide, train_y_slide, train_y_gan = sliding_window_torch(train_x, train_y_scaled, steps)
            test_x_slide, test_y_slide, test_y_gan = sliding_window_torch(test_x, test_y_scaled, steps)

            # -------------------------
            # WGAN-GP training
            # -------------------------
            batch_size = 128
            learning_rate1 = 0.000115
            learning_rate2 = 0.000115
            num_epochs = 60
            critic_iterations = 5

            train_dataset = DataLoader(
                TensorDataset(train_x_slide, train_y_gan),
                batch_size=batch_size,
                shuffle=True
            )

            generator = Generator(
                    input_size=train_x_slide.shape[2]  # n_features
                ).to(device)


            discriminator = Discriminator(
                sequence_length=train_y_gan.shape[1]     # steps+1
            ).to(device)

            optimizerG = optim.Adam(generator.parameters(), lr=learning_rate1, betas=(0.0, 0.9))
            optimizerD = optim.Adam(discriminator.parameters(), lr=learning_rate2, betas=(0.0, 0.9))

            histG = np.zeros(num_epochs)
            histD = np.zeros(num_epochs)

            for epoch in range(num_epochs):
                loss_D_epoch = []
                loss_G_epoch = []

                for xb, yb in train_dataset:
                    xb = xb.to(device)  # (batch, steps, n_features)
                    yb = yb.to(device)  # (batch, steps+1, 1)

                    # 1. Update Critic multiple times
                    for _ in range(critic_iterations):
                        optimizerD.zero_grad()

                        fake_last = generator(xb)  # (batch, 1)
                        fake_seq = torch.cat(
                            [yb[:, :steps, :],
                             fake_last.view(-1, 1, 1)],
                            dim=1
                        )  # (batch, steps+1, 1)

                        real_seq = yb

                        critic_real = discriminator(real_seq)
                        critic_fake = discriminator(fake_seq)

                        wasserstein_loss = -(critic_real.mean() - critic_fake.mean())
                        gp = gradient_penalty(discriminator, real_seq, fake_seq, device)

                        lossD = wasserstein_loss + lambda_gp * gp
                        lossD.backward()
                        optimizerD.step()

                    # 2. Update Generator once
                    optimizerG.zero_grad()
                    fake_last = generator(xb)
                    fake_seq = torch.cat(
                        [yb[:, :steps, :],
                         fake_last.view(-1, 1, 1)],
                        dim=1
                    )

                    critic_fake = discriminator(fake_seq)
                    lossG = -critic_fake.mean()
                    lossG.backward()
                    optimizerG.step()

                    loss_D_epoch.append(lossD.item())
                    loss_G_epoch.append(lossG.item())

                histD[epoch] = np.mean(loss_D_epoch)
                histG[epoch] = np.mean(loss_G_epoch)

                print(f"Epoch {epoch+1}/{num_epochs} | D loss: {histD[epoch]:.4f} | G loss: {histG[epoch]:.4f}")

            # -------------------------
            # Plot WGAN-GP losses
            # -------------------------
            plt.figure(figsize=(12, 6))
            plt.plot(histG, label='Generator Loss')
            plt.plot(histD, label='Discriminator Loss')
            plt.title(f'WGAN-GP Loss ({symbol}, steps={steps}, seed={seed})')
            plt.xlabel('Epochs')
            plt.legend(loc='upper right')
            plt.show()

            # -------------------------
            # Inference on train & test
            # -------------------------
            generator.eval()
            with torch.no_grad():
                pred_y_train = generator(train_x_slide.to(device)).cpu().numpy()
                pred_y_test = generator(test_x_slide.to(device)).cpu().numpy()

            y_train_true = y_scaler.inverse_transform(train_y_slide.numpy())
            y_train_pred = y_scaler.inverse_transform(pred_y_train)

            y_test_true = y_scaler.inverse_transform(test_y_slide.numpy())
            y_test_pred = y_scaler.inverse_transform(pred_y_test)

            # Training metrics & plot
            plt.figure(figsize=(12, 8))
            plt.plot(y_train_true, color='black', label='Actual Price')
            plt.plot(y_train_pred, color='blue', label='Predicted Price')
            plt.title(f'{symbol} Training prediction (steps={steps}, seed={seed})')
            plt.ylabel(symbol)
            plt.xlabel('Days')
            plt.legend(loc='upper left')
            plt.show()

            RMSE_train = mean_squared_error(y_train_true, y_train_pred, squared=False)
            acc_train = 1 - np.mean(np.abs((y_train_pred - y_train_true) / y_train_true))
            print(f'Training dataset RMSE: {RMSE_train}')
            print(f'Training dataset accuracy: {acc_train}')

            # Testing metrics & plot
            plt.figure(figsize=(12, 8))
            plt.plot(y_test_true, color='black', label='Actual Price')
            plt.plot(y_test_pred, color='blue', label='Predicted Price')
            plt.title(f'{symbol} Testing prediction (steps={steps}, seed={seed})')
            plt.ylabel(symbol)
            plt.xlabel('Days')
            plt.legend(loc='upper left')
            plt.show()

            RMSE_test = mean_squared_error(y_test_true, y_test_pred, squared=False)
            acc_test = 1 - np.mean(np.abs((y_test_pred - y_test_true) / y_test_true))
            print(f'Testing dataset RMSE: {RMSE_test}')
            print(f'Testing dataset accuracy: {acc_test}')

            # accumulate all results (global & per-run)
            re_train_error.append(RMSE_train)
            re_train_acc.append(acc_train)
            re_testing_error.append(RMSE_test)
            re_testing_acc.append(acc_test)

        

            # combined plot (train + test)
            plt.figure(figsize=(12, 6))
            plt.plot(np.arange(len(y_train_true)), y_train_true, color='black', label='Training Actual Price')
            plt.plot(np.arange(len(y_train_true)), y_train_pred, color='green', label='Training Predicted Price')

            offset = len(y_train_true)
            plt.plot(np.arange(offset, offset + len(y_test_true)), y_test_true,
                     color='black', label='Testing Actual Price')
            plt.plot(np.arange(offset, offset + len(y_test_true)), y_test_pred,
                     color='blue', label='Testing Predicted Price')

            plt.title(f'{symbol} prediction on training and testing (steps={steps}, seed={seed})')
            plt.xlabel('Time')
            plt.ylabel('Stock Price')
            plt.legend()
            plt.show()

        # -----------------------------
        # Save per-symbol, per-steps results to CSV
        # -----------------------------
        df = pd.DataFrame({
            "Seed": seeds,
            "Train Error": re_train_error,
            "Train Accuracy": re_train_acc,
            "Test Error": re_testing_error,
            "Test Accuracy": re_testing_acc
        })
        out_name = f"GRU-VAE-WGAN-without-VAE_results_{symbol}_{steps}.csv"
        df.to_csv(out_name, index=False)
        print(f"Results saved to {out_name}")
